<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Predicting shots made per game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties Ridge, Lasso, and Elastic Net to try and predict how many shots Kobe Bryant made per game in his career.

The Kobe shots dataset has hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model considering the limited number of observations (games) we have available. Furthermore, many of the predictors have significant multicollinearity. 

---

### 1. Load packages and data

In [4]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.cross_validation import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [5]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2. Examine the data

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why is this data that regularization might be particularly useful for?

In [20]:
# A:
kobe.columns.values

array(['SHOTS_MADE', 'AWAY_GAME', 'SEASON_OPPONENT:atl:1996-97',
       'SEASON_OPPONENT:atl:1997-98', 'SEASON_OPPONENT:atl:1999-00',
       'SEASON_OPPONENT:atl:2000-01', 'SEASON_OPPONENT:atl:2001-02',
       'SEASON_OPPONENT:atl:2002-03', 'SEASON_OPPONENT:atl:2003-04',
       'SEASON_OPPONENT:atl:2004-05', 'SEASON_OPPONENT:atl:2005-06',
       'SEASON_OPPONENT:atl:2006-07', 'SEASON_OPPONENT:atl:2007-08',
       'SEASON_OPPONENT:atl:2008-09', 'SEASON_OPPONENT:atl:2009-10',
       'SEASON_OPPONENT:atl:2010-11', 'SEASON_OPPONENT:atl:2011-12',
       'SEASON_OPPONENT:atl:2012-13', 'SEASON_OPPONENT:atl:2013-14',
       'SEASON_OPPONENT:atl:2014-15', 'SEASON_OPPONENT:atl:2015-16',
       'SEASON_OPPONENT:bkn:2012-13', 'SEASON_OPPONENT:bkn:2015-16',
       'SEASON_OPPONENT:bos:1996-97', 'SEASON_OPPONENT:bos:1997-98',
       'SEASON_OPPONENT:bos:1999-00', 'SEASON_OPPONENT:bos:2001-02',
       'SEASON_OPPONENT:bos:2002-03', 'SEASON_OPPONENT:bos:2003-04',
       'SEASON_OPPONENT:bos:2004-05', 

In [18]:
len(kobe.columns)

645

In [16]:
# Data regularization would be good for this dataset because there are so many columns/categories to choose from

In [ ]:
# SHOTS_MADE
# SHOT_TYPE:2pt_field_goal
# SHOT_TYPE:3pt_field_goal

---

### 3. Make predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [27]:
# A:

# Predictor X to include all columns except SHOTS_MADE; y is the target value of SHOTS_MADE
X = kobe.iloc[:,1:]
y = kobe.SHOTS_MADE.values

# Import Standard Scaler to begin standardizing features
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

# Use ss.fit_transform to standarize the values of X
Xss = ss.fit_transform(X)

In [ ]:
# Normalization is necessary for the regularized regressions because the predictor variable values and the beta values
# have to be evaluated on the same scale. 

---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [28]:
# A:
linreg = LinearRegression()

linreg_scores = cross_val_score(linreg, Xss, y, cv=10)

print linreg_scores
print np.mean(linreg_scores)

[ -1.43172380e+29  -2.53227878e+28  -3.33960462e+28  -2.62514467e+27
  -7.22387152e+27  -4.69173665e+27  -3.73187309e+27  -2.33035638e+27
  -5.25889200e+27  -5.29863145e+28]
-2.80739402702e+28


---

### 5. Find an optimal value for Ridge regression alpha using `RidgeCV`.

[Go to the documentation and read how RidgeCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

> *Hint: once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Ridge performs best searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit!


In [29]:
# A:
ridge_alphas = np.logspace(0, 5, 200)

optimal_ridge = RidgeCV(alphas = ridge_alphas, cv=10)

optimal_ridge.fit(Xss, y)

print optimal_ridge.alpha_

775.259748863


---

### 6. Cross-validate the Ridge regression $R^2$ with the optimal alpha.

Is it better than the Linear regression? If so, why might this be?

In [31]:
# A:
ridge = Ridge(alpha = optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, Xss, y, cv = 10)

print ridge_scores
print np.mean(ridge_scores)

[ 0.5822626   0.52880462  0.4869004   0.59583817  0.52396335  0.52942952
  0.52842192  0.41153448  0.39857643  0.44489752]
0.503062899682


---

### 7. Find an optimal value for Lasso regression alpha using `LassoCV`.

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) It is very similar to `RidgeCV`.

> *Hint: again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Lasso, unlike Ridge, performs best searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide itself what alphas to use by instead setting the keyword argument `n_alphas=` to however many alphas you want it to search over. It is recommended to let sklearn choose the range of alphas.

In [8]:
# A:
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(Xs, y)

print optimal_lasso.alpha_

---

### 8. Cross-validate the Lasso $R^2$ with the optimal alpha.

Is it better than the Linear regression? Is it better than Ridge? What do the differences in results imply about the issues with the dataset?

In [9]:
# A:
lasso = Lasso(alpha=optimal_lasso.alpha_)

lasso_scores = cross_val_score(lasso, Xs, y, cv=10)

print lasso_scores
print np.mean(lasso_scores)

---

### 9. Look at the coefficients for variables in the Lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original dataset are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** if you only fit the Lasso within `cross_val_score`, you will have to refit it outside of that
function to pull out the coefficients.

In [10]:
# A:

---

### 10. Find an optimal value for Elastic Net regression alpha using `ElasticNetCV`.

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note here that you will be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: strength of regularization
- `l1_ratio`: amount of ridge vs. lasso (0 = all ridge, 1 = all lasso)
    
Do not include 0 in the search for `l1_ratio`: it will not allow it and break!

You can use `n_alphas` for the alpha parameters instead of setting your own values: highly recommended!

Also - be careful setting too many l1_ratios over cross-validation folds in your search. It can take a very long time if you choose too many combinations and for the most part there are diminishing returns in this data.

In [11]:
# A:

---

### 11. Cross-validate the ElasticNet $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the Ridge and Lasso regularized regressions?

In [12]:
# A:

---

### 12. [Bonus] Compare the residuals for the Ridge and Lasso visually.


In [13]:
# A: